In [30]:
import pandas as pd
import numpy as np
from pathlib import Path
from slugify import slugify
import re

### Import CSV files

In [31]:
# path to raw data direcotry
web_scraping_dir = Path.cwd().parent
raw_data_dir = web_scraping_dir / "data" / "raw_data"

# open csv files
products_df = pd.read_csv(raw_data_dir / "products.csv", delimiter=";", parse_dates=["created_at"])
colors_df = pd.read_csv(raw_data_dir / "colors.csv", delimiter=";")
sizes_df = pd.read_csv(raw_data_dir / "sizes.csv", delimiter=";")
labels_df = pd.read_csv(raw_data_dir / "labels.csv", delimiter=";")
categories_df = pd.read_csv(raw_data_dir / "categories.csv", delimiter=";", parse_dates=["created_at"])
boots_category_df = (
    pd.read_csv(raw_data_dir / "boots_category.csv", delimiter=";")
    .drop("Product Type:", axis=1)
)
balls_category_df = (
    pd.read_csv( raw_data_dir / "balls_category.csv", delimiter=";")
    .drop("Product Type:", axis=1)
)

In [32]:
# Create images table by getting scraped images names
img_dir = web_scraping_dir / "data" / "product_images"
paths = img_dir.rglob("*")
dir_name = "product_images"
images = []

for path in paths:
    if dir_name in path.parts and path.is_file():
        prefix, img = str(path).split("/")[-2:]
        product_id = int(img.split("-")[0])
        image_name = f"{prefix}/{img}"
        is_thumbnail = True if int(img.split("-")[1][0]) == 1 else False

        row = {
            "product_id": product_id,
            "image_name": image_name,
            "is_thumbnail": is_thumbnail,
        }
        images.append(row)

images_df = pd.DataFrame(images)

### Helper functions

In [33]:
# Cleans text by removing HTML, quotes, and extra spaces.
def clean_description(data):
    data = re.sub(r"<.*?>", " ", data)
    data = re.sub(r'[,\'"]', "", data)
    data = re.sub(r"\s+", " ", data)
    return data.strip()[1:-1].strip()


# Formats column names: lowercase, underscores
def format_cols(df):
    col_names = {}
    for i, col in enumerate(df.columns):
        if i > 0:
            col_names[col] = col[:-1].lower().replace(" ", "_").replace("'", "")
        else:
            col_names[col] = col
    return col_names


# Strips and capitalizes specified string columns in a DataFrame.
def format_str(df, cols):
    for col in cols:
        df[col] = df[col].str.strip().str.title()

### Data cleaning

colors, sizes and categories are quite simple tables and don't require any transformation

In [34]:
# product table
products_df.drop_duplicates(inplace=True)
products_df["name"] = products_df["name"].str.split("\n", expand=True)[0]

products_df["price"] = products_df["price"].str.split("\n", expand=True)[1].str.strip().str[:-2].str.replace(",", ".").astype(float)
products_df["before_discount"] = (
    products_df["before_discount"]
    .str.split("\n", expand=True)[1]
    .str.strip()
    .str[:-2]
    .str.replace(",", ".")
    .astype(float)
)
products_df["sale_pct"] = round((
    products_df["before_discount"] - products_df["price"]
) / products_df["before_discount"] * 100)

products_df["description"] = products_df["description"].apply(clean_description)
products_df["slug"] = products_df["name"].apply(slugify)

previews = images_df.query("is_thumbnail == True")
products_df = (
    products_df.merge(previews, on="product_id", how="left")
    .drop(columns=["is_thumbnail"])
    .rename(columns={"image_name":"img_path"})
)

products_df.head(5)

,product_id,category_id,name,price,before_discount,description,scraped_num,scraped_link,created_at,sale_pct,slug,img_path
0,1,1,Cleats adidas F50 Elite LL FG - White,270.00,NaN,Boost your pace in lightweight adidas F50 shoe...,199578,https://www.r-gol.com/en/cleats-adidas-f50-eli...,2025-01-31,NaN,cleats-adidas-f50-elite-ll-fg-white,boots/1-1.jpg
1,2,1,Cleats Nike Zoom Mercurial Vapor 16 Elite FG -...,194.99,269.99,Do you have an obsession with speed? The bigge...,191711,https://www.r-gol.com/en/cleats-nike-zoom-merc...,2025-01-31,28.0,cleats-nike-zoom-mercurial-vapor-16-elite-fg-s...,boots/2-1.jpg
2,3,1,Cleats Nike Zoom Mercurial Vapor 16 Elite FG -...,182.99,269.99,Do you have an obsession with speed? The bigge...,191710,https://www.r-gol.com/en/cleats-nike-zoom-merc...,2025-01-31,32.0,cleats-nike-zoom-mercurial-vapor-16-elite-fg-b...,boots/3-1.jpg
3,4,1,adidas Mundial Team Boots - Black,107.99,150.00,This is probably one of the most popular shoe ...,95634,https://www.r-gol.com/en/adidas-mundial-team-b...,2025-01-31,28.0,adidas-mundial-team-boots-black,boots/4-1.jpg
4,5,1,Cleats Nike Zoom Mercurial Vapor 16 Elite FG -...,269.99,NaN,Do you have an obsession with speed? The bigge...,194479,https://www.r-gol.com/en/cleats-nike-zoom-merc...,2025-01-31,NaN,cleats-nike-zoom-mercurial-vapor-16-elite-fg-red,boots/5-1.jpg


In [35]:
from pprint import pprint
# pprint(products_df)
x = products_df[:10].values.flatten().tolist()
pprint(x)

[1,
 1,
 'Cleats adidas F50 Elite LL FG - White',
 270.0,
 nan,
 'Boost your pace in lightweight adidas F50 shoes designed for speed. These '
 'laceless Elite football boots feature a thin Fibertouch upper with a '
 'foot-hugging adidas PRIMEKNIT collar and 3D Sprintweb texture to keep the '
 'ball close to your foot. The responsive Sprintframe 360 outsole designed for '
 'playing on natural grass provides exceptional acceleration.',
 199578,
 'https://www.r-gol.com/en/cleats-adidas-f50-elite-ll-fg-white,p-199578',
 Timestamp('2025-01-31 00:00:00'),
 nan,
 'cleats-adidas-f50-elite-ll-fg-white',
 'boots/1-1.jpg',
 2,
 1,
 'Cleats Nike Zoom Mercurial Vapor 16 Elite FG - sky blue',
 194.99,
 269.99,
 'Do you have an obsession with speed? The biggest basketball stars have it '
 'too. Thats why we created the Elite cleats with an improved 3/4 length Air '
 'Zoom cushion. This model will provide you and the fastest players with the '
 'acceleration you need to break through the defense line.

In [36]:
labels_df["label_name"] = labels_df["label_name"].str.strip()
sizes_df["in_stock"] = np.where(sizes_df["in_stock"] == 1, True, False)
categories_df["category_name"] = np.where(
    categories_df["category_name"] == "football_boots", "boots", "balls"
)

In [37]:
# football boots category
boots_cols = format_cols(boots_category_df)
boots_category_df.rename(columns=boots_cols, inplace=True)
boots_cols_old = ["producer", "collections", "collection", "class", "upper", "color", "type_of_binding", "manufacturers_data"]
format_str(boots_category_df, boots_cols_old)

boots_category_df.rename(
    columns={"class": "class_type", "collection": "collection_name"}, inplace=True
)
boots_category_df.head(5)

,product_id,producer,collections,age_group,ground_type,class_type,upper,type_of_binding,boots_with_sock,collection_name,color,plays_in_these_boots,manufacturers_data,team
0,1,Adidas,Adidas F50,Men,Firm Ground (FG),Professional,Synthetic,Lack,No,Adidas Pure Victory,White,Lionel Messi,"Adidas Ag, Adi-Dassler-Strasse 1, 91074 Herzog...",NaN
1,2,Nike,Nike Mercurial Vapor,Men,Firm Ground (FG),Professional,Synthetic,Shoelace,No,Nike Mad Ambition Pack,Sky Blue,Robert Lewandowski,"Nike Retail B.V., Po Box 6453, Colosseum 1, 12...",NaN
2,3,Nike,Nike Mercurial Vapor,Men,Firm Ground (FG),Professional,Synthetic,Shoelace,No,Nike Shadow Pack 2024,Black,Robert Lewandowski,"Nike Retail B.V., Po Box 6453, Colosseum 1, 12...",NaN
3,4,Adidas,Adidas Classic,Men,Artificial Grass (AG/TF),Professional,Leather,Shoelace,No,Adidas Classic,Black,NaN,"Adidas Ag, Adi-Dassler-Strasse 1, 91074 Herzog...",NaN
4,5,Nike,Nike Mercurial Vapor,Men,Firm Ground (FG),Professional,Synthetic,Shoelace,No,Nike Mad Energy Pack,Red,Robert Lewandowski,"Nike Retail B.V., Po Box 6453, Colosseum 1, 12...",NaN


In [38]:
# football balls category
balls_cols = format_cols(balls_category_df)
balls_category_df.rename(columns=balls_cols, inplace=True)
balls_category_df["ball_size"] = (
    balls_category_df["ball_size"].replace("Futsal", 4).fillna(5).astype(int)
)
balls_cols_old = ["producer", "collection", "connecting_type", "manufacturers_data"]
format_str(balls_category_df, balls_cols_old)

balls_category_df.rename(
    columns={"weigh": "weight", "class": "class_type"}, inplace=True
)
balls_category_df.head(5)

,product_id,producer,ball_size,ground_type,class_type,collection,connecting_type,weight,color,manufacturers_data,league,team
0,453,Nike,5,Firm Ground (FG),Match,Other,Thermally Welded,410-450g,Yellow,"Nike Retail B.V., Po Box 6453, Colosseum 1, 12...",NaN,NaN
1,454,Adidas,5,Firm Ground (FG),Match,Ekstraklasa,NaN,410-450g,White,"Adidas Ag, Adi-Dassler-Strasse 1, 91074 Herzog...",NaN,NaN
2,455,Select,4,Indoor (IC/IN),Match,Select,Thermally Welded,400-440g,"White, Multicolor","Select Sport A/S, Fabriksparken 46, Dk 2600 Gl...",NaN,NaN
3,456,Adidas,5,Firm Ground (FG),Match,Ucl 24/25,Thermally Welded,410-450g,Multicolor,"Adidas Ag, Adi-Dassler-Strasse 1, 91074 Herzog...",NaN,NaN
4,457,Adidas,5,Firm Ground (FG),Match,Women'S Euro 2025,Thermally Welded,410-450g,White,"Adidas Ag, Adi-Dassler-Strasse 1, 91074 Herzog...",NaN,NaN


### Export cleaned data

In [39]:
cleaned_data_dir = web_scraping_dir / "data" / "cleaned_data"

products_df.to_csv(cleaned_data_dir / "products.csv", index=False, sep=";")
colors_df.to_csv(cleaned_data_dir / "colors.csv", index=False, sep=";")
sizes_df.to_csv(cleaned_data_dir / "sizes.csv", index=False, sep=";")
labels_df.to_csv(cleaned_data_dir / "labels.csv", index=False, sep=";")
categories_df.to_csv(cleaned_data_dir / "categories.csv", index=False, sep=";")
boots_category_df.to_csv(cleaned_data_dir / "boots_category.csv", index=False, sep=";")
balls_category_df.to_csv(cleaned_data_dir / "balls_category.csv", index=False, sep=";")
images_df.to_csv(cleaned_data_dir / "images.csv", index=False, sep=";")